In [1]:
import numpy as np

In [2]:
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
import matplotlib.patches as patches
def plot_rectangle(rec, ax=None):
    if ax is None: 
        ax = plt.subplot(aspect='equal')
    
    if isinstance(rec, (list, tuple)):
        for r in rec: 
            plot_rectangle(r,ax)
    
    else:
        size = (rec.maxes-rec.mins)
        ax.add_patch(patches.Rectangle(rec.mins, size[0], size[1], fill=False, zorder=-1))
    
    

In [4]:
#diff = np.float32(0.033068776)
diff = np.float32(0.03306878)
global_min = -31*diff
global_max = 31*diff

dom_maxs = np.array([global_max]*3, dtype=np.float64)
dom_mins = np.array([global_min]*3, dtype=np.float64)
tau = 0.2/12600 # 0.2 times mean interparticle separation
#buffer_tau = diff*5./150.
Ngrid = 62

In [5]:
import sparkhpc

In [6]:
ncores = 8

In [7]:
import os
os.environ['SPARK_HOME'] = '/home/ics/roskar/spark'
os.environ['SPARK_DRIVER_MEMORY'] = '16G'

In [8]:
import findspark
findspark.init()

In [9]:
import sparkhpc

In [10]:
# sj = sparkhpc.sparkjob.sparkjob(ncores=ncores,
#                                 memory=50000,
#                                 walltime='08:00', 
#                                 cores_per_executor=4, 
#                                 memory_per_executor=50000)

In [11]:
sj = sparkhpc.sparkjob.sparkjob(clusterid=0)

In [12]:
#sj = sparkhpc.sparkjob.LSFSparkJob(clusterid=0)

In [13]:
#sj.wait_to_start()

In [14]:
sc = sparkhpc.start_spark(master=sj.master_url, spark_conf='../conf', 
                          profiling=False, executor_memory='20000M', graphframes_package='graphframes:graphframes:0.3.0-spark2.0-s_2.11')

In [15]:
sc.setCheckpointDir('file:///zbox/data/roskar/work/checkpoint')

In [16]:
import spark_fof
reload(spark_fof.spark_fof)
reload(spark_fof)
path = '/zbox/trove/euclid/2Tlc-final/'

nMinMembers = 8
nBins = 62
minblock = 30
maxblock = 34
fof_analyzer = spark_fof.spark_fof.LCFOFAnalyzer(sc, path, nMinMembers, nBins, tau, dom_mins, dom_maxs, blockids=range(minblock,maxblock), buffer_tau=tau*2)

spark_fof: Number of input files:  64
spark_fof: Total number of particles:  4695434296


In [17]:
# sub-sample
fof_analyzer.particle_rdd = fof_analyzer.particle_rdd.map(lambda arr: np.random.choice(arr, np.int(np.ceil(len(arr)*0.1))), preservesPartitioning=True).cache()

In [18]:
fof_analyzer.particle_rdd.setName('particle_rdd')

particle_rdd PythonRDD[5] at RDD at PythonRDD.scala:48

In [19]:
%time fof_analyzer.particle_rdd.count()

CPU times: user 23 ms, sys: 8 ms, total: 31 ms
Wall time: 2min 27s


45887

In [20]:
fof_analyzer.fof_rdd.cache()

fof_rdd PythonRDD[14] at RDD at PythonRDD.scala:48

In [ ]:
%time fof_analyzer.fof_rdd.count()

In [ ]:
%time fof_analyzer.finalize_groups()

In [ ]:
import matplotlib
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
def convert_to_array(mem_usage): 
    times =[m[0][1] for m in  mem_usage]
    pids = np.sort(np.unique([m[0] for n in mem_usage for m in n[1:]]))
    
    time_lookup = {time: i for i,time in enumerate(times)}
    pid_lookup = {pid:i for i,pid in enumerate(pids)}
    
    data = np.zeros((len(times), len(pids)))
    
    for m in mem_usage:
        t = m[0][1]
        for pid,mem in m[1:]:
            data[time_lookup[t]][pid_lookup[pid]] = mem
    return data

In [ ]:
from cPickle import load
with open('/home/ics/roskar/mem.dump') as f: 
    mem_usage = load(f)
data = convert_to_array(mem_usage)

In [ ]:
f = plt.figure(figsize=(10,6))
for i in range(data.shape[1]):
    plt.plot(data[:,i])
plt.plot(data.sum(axis=1), '--')